In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
os.environ["POLARS_MAX_THREADS"] = "40"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
from miceforest import mean_match_shap,mean_match_default,mean_match_fast_cat
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
train_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(all_train_files[7])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["target","date"]),cs.contains("target"))
)
display(train_base.fetch().head())
train_base.collect(streaming=True).shape

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


(1526659, 5)

In [12]:
test_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.scan_parquet(all_test_files[33])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["date"]),cs.contains("target"))
)
display(test_base.fetch().head())
print(test_base.collect(streaming=True).shape)

case_id_base,month,week,weekday
u32,i8,i8,i8
57543,10,41,2
57549,10,41,2
57551,10,41,2
57552,10,41,3
57569,10,41,2


(10, 4)


In [14]:
train_files_df.filter(pl.col("filename").str.contains("appl"))

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


## Case_id, num_group1 and num_group2 relationships

<font size=6>

Goal of this competition is to predict risk quality of clients (or to be precise applications of clients), so client and his/her application is entity (case_id) for which your model predict probability of default (credit score).

- depth=0 (what we call static attributes) are attributes that are aggregated on case_id level. Example can be age of client or gender, 1 records per 1 case_id.

- depth=1, those are attributes where we have several records per client/application. Example can be previous applications or loans in credit bureau register, each client can have from 0 to n records. Therefore, for one case_id there might be several records, and to index them we use num_group1

- depth=2: For some attributes with depth=1 we have more detailed information, for example for previous applications we have data about instalments like date of payments or days past due of each payments. It means for each previous application, you can have 0…n records about instalments/payments. And as an index is used num_group2

To sum up, 1 client can have several previous applications, and each of those previous applications can have several records about instalments, payments, days past due, etc.

- num_group1 : 0 -> 19
- num_group2 : 0 -> 11

## Functions

In [15]:
def select_imputable_cols(df:pl.LazyFrame,threshold:int=10) -> pl.LazyFrame:
    col_list = (
        df
        .collect(streaming=True)
        .pipe(lambda df: (df.null_count()/df.shape[0])*100)
        .transpose(include_header=True,column_names=["percentage of nulls"])
        .join(
            feat_df,
            left_on="column",
            right_on="Variable",
            how="left"
        )
        .sort(by="percentage of nulls")
        .select(["column","Description","percentage of nulls"])
        .filter(pl.col("percentage of nulls") < threshold)
        ["column"].to_list()
    )
    _ = df.select(col_list).lazy()
    gc.collect()
    return _

In [16]:
def rename_cols(df: pl.LazyFrame) -> pl.LazyFrame:
    return (
        df
        .pipe(
            lambda df: df.rename(
                {_:_ + " " + feat_df.filter(pl.col("Variable") == _).select("Description").item() for _ in df.columns if _ not in ["case_id","num_group1","num_group2"]}
            )
        )
    )

In [17]:
train_previous_application = (
    pl.scan_parquet(path_to_train/"train_applprev_1_*.parquet",rechunk=True)
    .drop(cs.contains("date"))
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical),
    )
    .pipe(select_imputable_cols)
    .pipe(rename_cols)
)

In [18]:
train_previous_application.fetch().head(10)

case_id,num_group1,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,f32,f32
2,0,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,10000.00,0.00,"""CAL""","""CASH""",640.20,24.00,24.00
2,1,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,16000.00,0.00,"""CAL""","""CASH""",1682.40,12.00,12.00
3,0,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,11000.00,0.00,59999.80,0.00,"""CAL""","""CASH""",6140.00,12.00,12.00
4,0,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,16000.00,0.00,40000.00,0.00,"""CAL""","""CASH""",2556.60,24.00,24.00
5,0,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,62000.00,null,null,null,null,null,null,null,null
6,0,"""P94_109_143""","""P82_154_182""","""a55475b1""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,37000.00,0.00,15980.00,0.00,"""CAL""","""CASH""",1773.80,11.00,11.00
6,1,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,35000.00,0.00,32000.00,0.00,"""CAL""","""CASH""",4189.60,11.00,11.00
6,2,"""a55475b1""","""a55475b1""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,14000.00,0.00,17380.00,0.00,"""COL""","""POS""",1110.40,24.00,24.00
10,0,"""P73_130_169""","""P38_65_49""","""P17_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,57000.00,0.00,100000.00,0.00,"""CAL""","""CASH""",10916.60,12.00,12.00


In [19]:
train_previous_application_transformed = (
    train_previous_application
    .fill_null(strategy="mean")
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.float().max(),
        cs.categorical().mode().get(0)
    )
    .sort(by="case_id")
)
train_previous_application_transformed.fetch().head()

case_id,num_group1,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.
u32,u8,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
2,0,0.00,8200.00,0.00,16000.00,0.00,1682.40,24.00,24.00,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""CAL""","""CASH"""
3,0,0.00,11000.00,0.00,59999.80,0.00,6140.00,12.00,12.00,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""
4,0,0.00,16000.00,0.00,40000.00,0.00,2556.60,24.00,24.00,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
5,0,0.00,62000.00,2161.10,45335.69,471.80,3745.33,18.44,18.44,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
6,0,0.00,37000.00,0.00,32000.00,0.00,4189.60,24.00,24.00,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""


In [20]:
train_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


In [21]:
train_previous_application_personal = (
    pl.scan_parquet(all_train_files[0])
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains(["num_group"]).cast(pl.UInt8),
        cs.string().cast(pl.Categorical)
    )
    .pipe(select_imputable_cols)
    .pipe(rename_cols)
)
train_previous_application_personal.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,1,1,null
2,0,1,null
2,0,0,null
2,1,0,null
3,0,1,null


In [25]:
(
    train_previous_application_personal
    .collect(streaming=True)
    .filter(~cs.categorical().is_null())
    .head()
)

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
103059,6,0,"""a55475b1"""
103059,5,0,"""a55475b1"""
103059,7,0,"""a55475b1"""
103059,8,1,"""a55475b1"""
103059,1,1,"""a55475b1"""


In [109]:
train_previous_application_personal_transformed = (
    train_previous_application_personal
    .with_columns(cs.categorical().fill_null(cs.categorical().drop_nulls().mode().first()))
)
train_previous_application_personal_transformed.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,1,1,null
2,0,1,null
2,0,0,null
2,1,0,null
3,0,1,null
